In [43]:
%reload_ext sql
%sql sqlite:///health.db

In [47]:
import sqlite3
conn = sqlite3.connect('health.db')
c = conn.cursor()

c.executescript('''
DROP TABLE IF EXISTS medications;
DROP TABLE IF EXISTS diagnoses;
DROP TABLE IF EXISTS visits;
DROP TABLE IF EXISTS patients;
DROP TABLE IF EXISTS doctors;
                
CREATE TABLE patients (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    age INTEGER,
    gender TEXT CHECK(gender IN ('Male','Female','Other')),
    phone TEXT UNIQUE,
    blood_type TEXT,
    created_at TEXT DEFAULT (datetime('now'))
);

CREATE TABLE visits (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    patient_id INTEGER,
    visit_date TEXT DEFAULT (date('now')),
    doctor TEXT,
    reason TEXT,
    cost REAL DEFAULT 0,
    FOREIGN KEY(patient_id) REFERENCES patients(id)
);

CREATE TABLE diagnoses (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    visit_id INTEGER,
    disease TEXT,
    severity TEXT CHECK(severity IN ('Mild','Moderate','Severe','Critical')),
    FOREIGN KEY(visit_id) REFERENCES visits(id)
);

CREATE TABLE medications (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    diagnosis_id INTEGER,
    drug_name TEXT,
    dosage TEXT,
    duration_days INTEGER,
    FOREIGN KEY(diagnosis_id) REFERENCES diagnoses(id)
);
CREATE TABLE doctors (
    id INTEGER PRIMARY KEY,
    name TEXT UNIQUE,
    specialty TEXT
);
                
INSERT INTO patients (name, age, gender, phone, blood_type) VALUES
('Sudip Adhikari', 22, 'Male', '9876543210', 'O+'),
('Ram Khadka', 38, 'Male', '9123456789', 'A+'),
('Shyam Magar', 29, 'Male', '9988776655', 'B+'),
('Ritika KC', 61, 'Female', '8765432109', 'AB-'),
('Rajaram Poudel', 45, 'Male', '7654321098', 'O-'),
('Puja Mahato', 33, 'Female', '8877665544', 'A-'),
('Rohit Paudel', 27, 'Male', '9988112233', 'B-');

INSERT INTO visits (patient_id, visit_date, doctor, reason, cost) VALUES
(1, '2025-03-15', 'Dr. Adhikari', 'Diabetes & BP Check', 1200),
(1, '2025-04-10', 'Dr. Adhikari', 'Follow-up', 600),
(2, '2025-03-20', 'Dr. KC', 'Thyroid Issue', 900),
(3, '2025-04-01', 'Dr. Adhikari', 'Viral Fever', 800),
(4, '2025-04-12', 'Dr. KC', 'Joint Pain', 1500),
(5, '2025-04-15', 'Dr. Dhakal', 'Hypertension', 1100),
(6, '2025-04-20', 'Dr. Adhikari', 'Anemia Check', 700);

INSERT INTO diagnoses (visit_id, disease, severity) VALUES
(1, 'Type 2 Diabetes', 'Moderate'),
(1, 'Hypertension', 'Moderate'),
(2, 'Hypothyroidism', 'Mild'),
(3, 'Viral Fever', 'Mild'),
(5, 'Hypertension', 'Severe'),
(6, 'Osteoarthritis', 'Moderate'),
(7, 'Iron Deficiency Anemia', 'Moderate');

INSERT INTO medications (diagnosis_id, drug_name, dosage, duration_days) VALUES
(1, 'Metformin', '500mg twice daily', 180),
(2, 'Amlodipine', '5mg daily', 365),
(3, 'Levothyroxine', '100mcg daily', 365),
(5, 'Losartan', '50mg daily', 180),
(6, 'Diclofenac Gel', 'Apply twice daily', 30),
(7, 'Ferrous Ascorbate', '100mg daily', 90);
                
INSERT INTO doctors VALUES
(1, 'Dr. Adhikari', 'Cardiologist'),
(2, 'Dr. KC', 'Endocrinologist'),
(3, 'Dr. Dhakal', 'Orthopedic'),
(4, 'Dr. Bhatta', 'General');
''');



## To select all patients name

In [48]:
%%sql
SELECT * FROM patients;

 * sqlite:///health.db
Done.


id,name,age,gender,phone,blood_type,created_at
1,Sudip Adhikari,22,Male,9876543210,O+,2025-12-05 04:58:15
2,Ram Khadka,38,Male,9123456789,A+,2025-12-05 04:58:15
3,Shyam Magar,29,Male,9988776655,B+,2025-12-05 04:58:15
4,Ritika KC,61,Female,8765432109,AB-,2025-12-05 04:58:15
5,Rajaram Poudel,45,Male,7654321098,O-,2025-12-05 04:58:15
6,Puja Mahato,33,Female,8877665544,A-,2025-12-05 04:58:15
7,Rohit Paudel,27,Male,9988112233,B-,2025-12-05 04:58:15


## To select name and age from patients 

In [49]:
%%sql
SELECT name, age FROM patients;

 * sqlite:///health.db
Done.


name,age
Sudip Adhikari,22
Ram Khadka,38
Shyam Magar,29
Ritika KC,61
Rajaram Poudel,45
Puja Mahato,33
Rohit Paudel,27


## To see the total number of patients

In [20]:
%%sql
SELECT COUNT(*) AS total_patients FROM patients;

 * sqlite:///health.db
Done.


total_patients
7


## To select patients older than 40

In [21]:
%%sql
SELECT name, age, phone FROM patients WHERE age > 40;

 * sqlite:///health.db
Done.


name,age,phone
Ritika KC,61,8765432109
Rajaram Poudel,45,7654321098


## To select female patients

In [22]:
%%sql
SELECT name, blood_type, age FROM patients WHERE gender = 'Female';

 * sqlite:///health.db
Done.


name,blood_type,age
Ritika KC,AB-,61
Puja Mahato,A-,33


## To select patients with O+ blood group

In [35]:
%%sql
SELECT name FROM patients WHERE blood_type = 'O+';

 * sqlite:///health.db
Done.


name
Sudip Adhikari


### To sort patients by age(youngest)

In [25]:
%%sql
SELECT name, age FROM patients ORDER BY age ASC;

 * sqlite:///health.db
Done.


name,age
Sudip Adhikari,22
Rohit Paudel,27
Shyam Magar,29
Puja Mahato,33
Ram Khadka,38
Rajaram Poudel,45
Ritika KC,61


### All visits with patients name (USING INNER JOIN)

In [37]:
%%sql
SELECT p.name, v.visit_date, v.doctor, v.reason 
FROM visits v 
Inner JOIN patients p ON v.patient_id = p.id;

 * sqlite:///health.db
Done.


name,visit_date,doctor,reason
Sudip Adhikari,2025-03-15,Dr. Adhikari,Diabetes & BP Check
Sudip Adhikari,2025-04-10,Dr. Adhikari,Follow-up
Ram Khadka,2025-03-20,Dr. KC,Thyroid Issue
Shyam Magar,2025-04-01,Dr. Adhikari,Viral Fever
Ritika KC,2025-04-12,Dr. KC,Joint Pain
Rajaram Poudel,2025-04-15,Dr. Dhakal,Hypertension
Puja Mahato,2025-04-20,Dr. Adhikari,Anemia Check


### Total Visits per Patient

In [27]:
%%sql
SELECT p.name, COUNT(v.id) AS visit_count
FROM patients p
LEFT JOIN visits v ON p.id = v.patient_id
GROUP BY p.name;

 * sqlite:///health.db
Done.


name,visit_count
Puja Mahato,1
Rajaram Poudel,1
Ram Khadka,1
Ritika KC,1
Rohit Paudel,0
Shyam Magar,1
Sudip Adhikari,2


### Patients Who Visited Dr. Adhikari

In [28]:
%%sql
SELECT DISTINCT p.name
FROM patients p
JOIN visits v ON p.id = v.patient_id
WHERE v.doctor = 'Dr. Adhikari';

 * sqlite:///health.db
Done.


name
Sudip Adhikari
Shyam Magar
Puja Mahato


### All Diagnoses with Patient & Doctor

In [29]:
%%sql
SELECT p.name, d.disease, d.severity, v.doctor
FROM diagnoses d
JOIN visits v ON d.visit_id = v.id
JOIN patients p ON v.patient_id = p.id;

 * sqlite:///health.db
Done.


name,disease,severity,doctor
Sudip Adhikari,Type 2 Diabetes,Moderate,Dr. Adhikari
Sudip Adhikari,Hypertension,Moderate,Dr. Adhikari
Sudip Adhikari,Hypothyroidism,Mild,Dr. Adhikari
Ram Khadka,Viral Fever,Mild,Dr. KC
Ritika KC,Hypertension,Severe,Dr. KC
Rajaram Poudel,Osteoarthritis,Moderate,Dr. Dhakal
Puja Mahato,Iron Deficiency Anemia,Moderate,Dr. Adhikari


### Full Medication History

In [30]:
%%sql
SELECT p.name AS patient, m.drug_name, m.dosage, m.duration_days, d.disease
FROM medications m
JOIN diagnoses d ON m.diagnosis_id = d.id
JOIN visits v ON d.visit_id = v.id
JOIN patients p ON v.patient_id = p.id;

 * sqlite:///health.db
Done.


patient,drug_name,dosage,duration_days,disease
Sudip Adhikari,Metformin,500mg twice daily,180,Type 2 Diabetes
Sudip Adhikari,Amlodipine,5mg daily,365,Hypertension
Sudip Adhikari,Levothyroxine,100mcg daily,365,Hypothyroidism
Ritika KC,Losartan,50mg daily,180,Hypertension
Rajaram Poudel,Diclofenac Gel,Apply twice daily,30,Osteoarthritis
Puja Mahato,Ferrous Ascorbate,100mg daily,90,Iron Deficiency Anemia


### Most Common Disease

In [31]:
%%sql
SELECT disease, COUNT(*) AS cases
FROM diagnoses
GROUP BY disease
ORDER BY cases DESC;

 * sqlite:///health.db
Done.


disease,cases
Hypertension,2
Viral Fever,1
Type 2 Diabetes,1
Osteoarthritis,1
Iron Deficiency Anemia,1
Hypothyroidism,1


### Severe Cases Only

In [32]:
%%sql
SELECT p.name, d.disease
FROM diagnoses d
JOIN visits v ON d.visit_id = v.id
JOIN patients p ON v.patient_id = p.id
WHERE d.severity = 'Severe';

 * sqlite:///health.db
Done.


name,disease
Ritika KC,Hypertension


### Patients with Multiple Visits

In [33]:
%%sql
SELECT p.name, COUNT(v.id) AS visits
FROM patients p
JOIN visits v ON p.id = v.patient_id
GROUP BY p.id, p.name
HAVING visits > 1;

 * sqlite:///health.db
Done.


name,visits
Sudip Adhikari,2


### Most Expensive Visit

In [34]:
%%sql
SELECT p.name, v.cost, v.reason
FROM visits v
JOIN patients p ON v.patient_id = p.id
ORDER BY v.cost DESC
LIMIT 1;

 * sqlite:///health.db
Done.


name,cost,reason
Ritika KC,1500.0,Joint Pain


### RIGHT JOIN 

In [53]:
%%sql
SELECT 
    COALESCE(p.name, 'Unknown Patient') AS patient,
    v.doctor,
    v.visit_date,
    v.reason
FROM visits v
LEFT JOIN patients p ON v.patient_id = p.id;

 * sqlite:///health.db
Done.


patient,doctor,visit_date,reason
Sudip Adhikari,Dr. Adhikari,2025-03-15,Diabetes & BP Check
Sudip Adhikari,Dr. Adhikari,2025-04-10,Follow-up
Ram Khadka,Dr. KC,2025-03-20,Thyroid Issue
Shyam Magar,Dr. Adhikari,2025-04-01,Viral Fever
Ritika KC,Dr. KC,2025-04-12,Joint Pain
Rajaram Poudel,Dr. Dhakal,2025-04-15,Hypertension
Puja Mahato,Dr. Adhikari,2025-04-20,Anemia Check


### Cross join

In [ ]:
%%sql
SELECT 
    p.name AS patient,
    d.name AS doctor,
    d.specialty
FROM (SELECT name FROM patients LIMIT 4) p  
CROSS JOIN doctors d
ORDER BY p.name, d.name;

 * sqlite:///health.db
Done.


patient,doctor,specialty
Ram Khadka,Dr. Adhikari,Cardiologist
Ram Khadka,Dr. Bhatta,General
Ram Khadka,Dr. Dhakal,Orthopedic
Ram Khadka,Dr. KC,Endocrinologist
Ritika KC,Dr. Adhikari,Cardiologist
Ritika KC,Dr. Bhatta,General
Ritika KC,Dr. Dhakal,Orthopedic
Ritika KC,Dr. KC,Endocrinologist
Shyam Magar,Dr. Adhikari,Cardiologist
Shyam Magar,Dr. Bhatta,General


### Self Join

In [55]:
%%sql
SELECT 
    p.name AS patient,
    v1.doctor,
    v1.visit_date AS first_visit,
    v2.visit_date AS second_visit
FROM visits v1
JOIN visits v2 ON v1.patient_id = v2.patient_id 
              AND v1.doctor = v2.doctor 
              AND v1.visit_date < v2.visit_date
JOIN patients p ON v1.patient_id = p.id
ORDER BY p.name;

 * sqlite:///health.db
Done.


patient,doctor,first_visit,second_visit
Sudip Adhikari,Dr. Adhikari,2025-03-15,2025-04-10
